In [1]:
import json
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

with open('Synth_data/normalised_intent_validation_slotfixed_set.json', 'r') as file:
    data = json.load(file)

# Extract utterances and their associated domain (hotel or train)
utterances = []
labels = []

for scenario in data:
    for dialogue in scenario['scenarios']:
        dialogue_service = None
        for turn in dialogue['turns']:
            if turn['frames']:  # Check if there are any frames to define service
                dialogue_service = turn['frames'][0]['service']  # Take the service from the first frame
            if turn["speaker"] == "USER":  # Assuming user turns are from "participant1"
                utterances.append(turn['utterance'])
                # label the turn based on the service
                if dialogue_service in ["hotel", "train"]:
                    labels.append(dialogue_service)
                else:
                    # 'other' if the service is not one of the target domains
                    labels.append("other")

# Filter out 'other' labels 
filtered_utterances_validation = [utterance for utterance, label in zip(utterances, labels) if label in ["hotel", "train"]]
filtered_labels_validation = [label for label in labels if label in ["hotel", "train"]]

# Create a DataFrame
df_validated = pd.DataFrame({'utterance': filtered_utterances_validation, 'domain': filtered_labels_validation})


In [2]:
# Load the dataset
with open('Synth_data/normalised_intent_train_slotfixed_set.json', 'r') as file:
    data = json.load(file)

# Extract utterances and their associated domain (hotel or train)
utterances = []
labels = []

for scenario in data:
    for dialogue in scenario['scenarios']:
        dialogue_service = None
        for turn in dialogue['turns']:
            if turn['frames']:  # Check if there are any frames to define service
                dialogue_service = turn['frames'][0]['service']  # Take the service from the first frame
            if turn["speaker"] == "USER":  # Assuming user turns are from "participant1"
                utterances.append(turn['utterance'])
                # label the turn based on the service
                if dialogue_service in ["hotel", "train"]:
                    labels.append(dialogue_service)
                else:
                    # 'other' if the service is not one of the target domains
                    labels.append("other")

# Filter out 'other' labels 
filtered_utterances_train = [utterance for utterance, label in zip(utterances, labels) if label in ["hotel", "train"]]
filtered_labels_train = [label for label in labels if label in ["hotel", "train"]]

# Create a DataFrame
df_train = pd.DataFrame({'utterance': filtered_utterances_train, 'domain': filtered_labels_train})

In [3]:
# Load the dataset
with open('Synth_data/normalised_intent_test_slotfixed_set.json', 'r') as file:
    data = json.load(file)

# Extract utterances and their associated domain (hotel or train)
utterances = []
labels = []

for scenario in data:
    for dialogue in scenario['scenarios']:
        dialogue_service = None
        for turn in dialogue['turns']:
            if turn['frames']:  # Check if there are any frames to define service
                dialogue_service = turn['frames'][0]['service']  # Take the service from the first frame
            if turn["speaker"] == "USER":  # Assuming user turns are from "participant1"
                utterances.append(turn['utterance'])
                # label the turn based on the service
                if dialogue_service in ["hotel", "train"]:
                    labels.append(dialogue_service)
                else:
                    # 'other' if the service is not one of the target domains
                    labels.append("other")

# Filter out 'other' labels 
filtered_utterances_test = [utterance for utterance, label in zip(utterances, labels) if label in ["hotel", "train"]]
filtered_labels_test = [label for label in labels if label in ["hotel", "train"]]

# Create a DataFrame
df_test = pd.DataFrame({'utterance': filtered_utterances_test, 'domain': filtered_labels_test})

In [4]:
print(df_train)

                                              utterance domain
0     Can you tell me the train schedule and how muc...  train
1     Yes, I would like to purchase a ticket for the...  train
2     No, that's all I needed. Thank you for your help!  train
3     Actually, I just remembered, can I add a retur...  train
4     Can you tell me the train schedule and how to ...  train
...                                                 ...    ...
2937  No, that's all for now. Thank you for the info...  hotel
2938  Thank you! I have a reservation and would like...  hotel
2939  Yes, please. Could you give me more details ab...  hotel
2940  Yes, that would be great. Could you arrange a ...  hotel
2941  9 AM would be perfect. Thank you for arranging...  hotel

[2942 rows x 2 columns]


In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir='BERT_cache_folder')

def encode_data(tokenizer, texts, max_length=128):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

# Encode utterances
encoded_inputs_validate = encode_data(tokenizer, df_validated['utterance'].tolist())


In [6]:
encoded_inputs_test = encode_data(tokenizer, df_test['utterance'].tolist())
encoded_inputs_train = encode_data(tokenizer, df_train['utterance'].tolist())

In [7]:
# Unique domain mapping to integers
domain_labels = {domain: idx for idx, domain in enumerate(df_validated['domain'].unique())}
df_validated['label'] = df_validated['domain'].map(domain_labels)

# Prepare labels
labels = df_validated['label'].values


In [8]:
class UtteranceDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset_validate = UtteranceDataset(encoded_inputs_validate, labels)
dataset_test = UtteranceDataset(encoded_inputs_test, labels)
dataset_train = UtteranceDataset(encoded_inputs_train, labels)


In [9]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU instead.")


CUDA is available. Using GPU: NVIDIA GeForce GTX 1080 Ti


In [10]:
from transformers import TrainerCallback, TrainerState, TrainerControl
from tqdm.auto import tqdm

class PrintCallback(TrainerCallback):
    def __init__(self):
        self.progress_bar = None

    def on_train_begin(self, args, state, control, **kwargs):
        print("Starting training...")
        self.progress_bar = tqdm(total=state.num_train_epochs)

    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\nEpoch {state.epoch + 1}/{state.num_train_epochs}")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if 'loss' in logs:
                print(f"  Training loss: {logs['loss']:.4f}")
            if 'eval_loss' in logs:
                print(f"  Validation loss: {logs['eval_loss']:.4f}")
            if 'eval_accuracy' in logs:
                print(f"  Validation accuracy: {logs['eval_accuracy']:.4f}")

    def on_epoch_end(self, args, state, control, **kwargs):
        self.progress_bar.update(1)
        if 'eval_loss' in logs:
            print(f"End of epoch {state.epoch + 1}. Evaluation loss: {logs['eval_loss']:.4f}")
        if 'eval_accuracy' in logs:
            print(f"  Validation accuracy: {logs['eval_accuracy']:.4f}")

    def on_train_end(self, args, state, control, **kwargs):
        self.progress_bar.close()
        print("Training completed.")


In [11]:
import torch

print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

if torch.cuda.is_available():
    print("Number of CUDA devices:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")


Is CUDA available: True
CUDA version: 12.1
Number of CUDA devices: 4
Device 0: NVIDIA GeForce GTX 1080 Ti
Device 1: NVIDIA GeForce GTX 1080 Ti
Device 2: NVIDIA GeForce GTX 1080 Ti
Device 3: NVIDIA GeForce GTX 1080 Ti


In [12]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)  # Replace 0 with the device number you want to use
    print(f"Default CUDA device set: {torch.cuda.current_device()}")


Default CUDA device set: 0


In [13]:
!nvidia-smi

Wed May  1 12:46:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:03:00.0 Off |                  N/A |
| 51%   62C    P8              19W / 250W |   3554MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(domain_labels))

model.to(device) 


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [22]:
import torch
from torch.utils.data import DataLoader
from transformers import get_scheduler, BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
import math

# Assuming model, train_dataset, validation_dataset, and tokenizer are predefined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Specify the number of classes explicitly
num_classes = 2  # Since you have 'hotel' and 'train'

# Convert labels to a NumPy array for easier manipulation
labels_np = np.array(filtered_labels_train)

# Calculate class counts
num_hotel = np.sum(labels_np == 'hotel')
num_train = np.sum(labels_np == 'train')

# Calculate class weights
class_counts = [num_hotel, num_train]
total_counts = sum(class_counts)
# Existing weight calculation
class_weights = torch.tensor([total_counts / class_count for class_count in class_counts], dtype=torch.float).to(device)

# Let's say you want to give more weight to 'hotel' which is the first class in your list
# You can multiply the weight for 'hotel' by a factor, e.g., 2, 3, or any factor you deem appropriate
factor = 3  # Experiment with this factor
class_weights[0] *= factor

print("Adjusted Class Weights:", class_weights)

# Convert datasets to DataLoader
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
validation_loader = DataLoader(dataset_validate, batch_size=32)

# Setting up the optimizer
optimizer = AdamW(model.parameters(), lr=0.001)

# Total number of training steps
num_training_steps = len(train_loader) * 10  # Adjust epochs as needed

# Create the learning rate scheduler.
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=num_training_steps
)

# Best validation loss tracking
best_validation_loss = float('inf')
model_path = "./model_save_domain_synth_temp"

# Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(10):  # loop over the dataset multiple times
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        labels = batch['labels']
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, num_classes), labels.view(-1))

        # Backpropagation
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Validation step at the end of each epoch
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in validation_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            labels = batch['labels']
            loss = loss_fct(logits.view(-1, num_classes), labels.view(-1))
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(validation_loader)
    print(f"Validation Loss: {avg_val_loss}")

    # Check if this is the best validation loss encountered and save model if it is
    if avg_val_loss < best_validation_loss:
        best_validation_loss = avg_val_loss
        print(f"New Best Validation Loss: {best_validation_loss}")
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)

progress_bar.close()

# Optionally save the model and tokenizer again at the end if needed
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)


Adjusted Class Weights: tensor([6.2024, 1.9368], device='cuda:0')
Exponentially Adjusted Class Weights: tensor([6.2024, 1.9368], device='cuda:0')


  3%|████                                                                                                                    | 2/60 [00:00<00:12,  4.48it/s]

Epoch 1, Loss: 0.6418927311897278


  5%|██████                                                                                                                  | 3/60 [00:00<00:16,  3.49it/s]

Epoch 1, Loss: 0.5069409012794495


...

Epoch 10, Loss: 0.06744872033596039


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:31<00:00,  2.51it/s]

Epoch 10, Loss: 0.18293148279190063
Epoch 10, Loss: 0.08297879993915558


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:32<00:00,  1.83it/s]

Validation Loss: 1.472532684604327


In [14]:
import torch
from torch.utils.data import DataLoader
from transformers import get_scheduler
from torch.optim import AdamW
from tqdm import tqdm

# Assuming model, train_dataset, validation_dataset, and tokenizer are predefined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Convert datasets to DataLoader
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
validation_loader = DataLoader(dataset_validate, batch_size=32)

# Setting up the optimizer
optimizer = AdamW(model.parameters(), lr=0.001)

# Total number of training steps
num_training_steps = len(train_loader) * 10  # num_epochs is 15

# Create the learning rate scheduler.
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=num_training_steps
)

# Best validation loss tracking
best_validation_loss = float('inf')
model_path = "./model_save_domain_synth"

# Training loop
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(10):  # loop over the dataset multiple times
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Validation step at the end of each epoch
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in validation_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(validation_loader)
    print(f"Validation Loss: {avg_val_loss}")

    # Check if this is the best validation loss encountered and save model if it is
    if avg_val_loss < best_validation_loss:
        best_validation_loss = avg_val_loss
        print(f"New Best Validation Loss: {best_validation_loss}")
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)

progress_bar.close()




# Save the model
model_path = "./model_save_domain_synth"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


  2%|██                                                                                                                      | 1/60 [00:00<00:52,  1.12it/s]

Epoch 1, Loss: 0.6948522925376892


  5%|██████                                                                                                                  | 3/60 [00:01<00:21,  2.61it/s]

Epoch 1, Loss: 0.7671108245849609


  7%|████████                                                                                                                | 4/60 [00:01<00:21,  2.67it/s]

...

 45%|█████████████████████████████████████████████████████▌                                                                 | 27/60 [00:17<00:15,  2.12it/s]

Epoch 5, Loss: 0.6458950042724609


 47%|███████████████████████████████████████████████████████▌                                                               | 28/60 [00:17<00:13,  2.29it/s]

Epoch 5, Loss: 0.6435486078262329


KeyboardInterrupt: 

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
test_loader = DataLoader(dataset_test, batch_size=32) 

model.eval()  # Set model to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

predictions, true_labels = [], []

# Evaluate the model
with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.detach().cpu().numpy())
        true_labels.extend(labels.detach().cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')  # adjust 'average' as needed

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.4888888888888889
Precision: 0.49375
Recall: 0.49377799900447983
F1 Score: 0.48882578096061247
